# Collaboration and Competition Report

## Environment Details

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1. If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01. Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Each agent receives its own, local observation. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping.

The task is episodic, and in order to solve the environment, your agents must get an average score of +0.5 (over 100 consecutive episodes, after taking the maximum over both agents). Specifically,

After each episode, we add up the rewards that each agent received (without discounting), to get a score for each agent. This yields 2 (potentially different) scores. We then take the maximum of these 2 scores.
This yields a single score for each episode.
The environment is considered solved, when the average (over 100 episodes) of those scores is at least +0.5.



## Learning Algorithm

Two agents must collaborate in this enviornment to maximize reward.  According to the [MADDPG Paper](https://arxiv.org/abs/1706.02275) we need to make additional considerations when working with multiple agents.

> Unfortunately, traditional reinforcement learning approaches such as Q-Learning or policy gradient
are poorly suited to multi-agent environments. One issue is that each agent’s policy is changing
as training progresses, and the environment becomes non-stationary from the perspective of any
individual agent (in a way that is not explainable by changes in the agent’s own policy). This presents
learning stability challenges and prevents the straightforward use of past experience replay

After reading this, **I believe a single agent for this project is sufficient because changes ARE explainable by changes to the agent's own policy**.  I tried both single agent self play, and multi agent cooperation using MADDPG.  In the case of multi agent, both agents shared a common replay buffer as indicated in the MADDPG paper.  After trying both, I found that a single agent performed just as well, so I opted to use the simpler [DDPG](https://arxiv.org/pdf/1509.02971.pdf) with single agent self play.

![DDPG](images/DDPG.png)


### Exploration

I had a tough time getting the agent to learn consistantly on this project.  One issue I ran into was the initial exploration of the environment. The tennis rackets never hit the ball accidentally.  To tackle this, I tried a few different things starting with adjusting the parameters of the Ornstein-Uhlenbeck process.  Actually, I was never able to get Ornstein-Uhlenbeck noise to perform well. My agent ended up getting stuck in a behavior that did not lead to any reward, repeating it over and over. There are some very specific seeds that produced good results, but I wanted my agent to learn without specifying a seed.   One thing I did that ended up being pretty successful is using a simpler gaussian noise function instead. With my simple noise function, the agents quickly hit the ball a few times and were able to get a little bit of signal to learn towards.

The next thing I implemented was a prioritized replay buffer.  I figured that since we are dealing with sparse rewards, those experiences will have high error, and we will learn from those good experiences more often.  This coupled with a simple noise function produced good repeatable results for me.  I addapted my code from a nice example of the Prioritized Replay Buffer found here: https://github.com/higgsfield/RL-Adventure.


### Hyperparameters.

The actor and critic networks use soft update for target networks as outlined in the DDPG algorithm.  I found a relaitvely larger $tau$ parameter at $1e^{-2}$ produced good results.

```
BUFFER_SIZE = 1e5
BATCH_SIZE = 128
GAMMA = 0.99
TAU = 1e-2
LR_ACTOR = 1e-3
LR_CRITIC = 1e-3
```

### Networks

Each agent contains an actor and critic network.  After exprimenting with different hidden layers, I made a couple updates.

- Batch normalization on critic network improved learning.
- Hidden layers of 256 & 128 seemed to produce learning of more complicated behavior.  Using less hidden units, my agents did not learn what to do after the initial hit of the ball, and using more hidden units did not seem to improve performance much.


#### Actor Network

![Actor](images/actor.png)

#### Critic Network

![Critic](images/critic.png)

I emulated my network diagrams from some nice diagrams I found on another student's submission: https://github.com/brinij/p3_collab-compet/blob/master/Report.md.  The author indicated she used [draw.io](https://draw.io) to make this drawing and I was able to make something that looked pretty similar in just a few minutes.


## Plot of Rewards

**Solved in 164 episodes!**

![Plot](images/plot.png)

## Trained Agent

![Trained Agent](images/trained.gif)

## Ideas for Future Work

As always, I think we can tune network and hyperparameters, but I think exploration challenges set the tennis enviroment apart from the previos projects.  My improvement ideas center around how to overcome the sparse rewards of this environment.

- [Curiosity Driven Exploration](https://pathak22.github.io/noreward-rl/resources/icml17.pdf) to tackle sparse rewards of the tennis environment. I attempted to implement the ICM module in this project but ended up either not implementing correctly or not tuning the paremeters correctly because it failed to ever hit the ball with ICM in place.  Additional work here to debug or tune parameters might produce different results.
- [Hindsight Experience Replay](https://arxiv.org/abs/1707.01495) is another potential solution to working with sparse rewards.  To incorporate HER, I would feature engineer a goal state where the racket is touching the ball, and see if that encourages the initial hitting of the ball.